<a href="https://colab.research.google.com/github/stebechoi/CP2/blob/Hwkdir/2.%20(MovieLense100K)_CB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MovieLense 데이터를 이용한 영화 추천 시스템 (CB)

## 추천시스템(recommender system)
- 사용자의 과거 행동 데이터나 다른 데이터를 바탕으로 사용자에게 필요한 정보나 제품을 골라서 제시해 주는 시스템

- 내용 기반 필터링 (Content-Based Filtering) </br>
  : 제품의 내용을 분석해서 추천
    - 영화추천의 경우, 장르, 배우, 감독, 줄거리 등을 활용할 수 있다.

### 데이터 로드

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/CP/CP2')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('movielense_rt100.csv')
df

,user_id,movie_id,movie_title,rating,TotalRatingCount,release_year,release_month,rating_year,rating_month,age_group,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,308,1,Toy Story (1995),4,452,1995,1,1998,2,6,...,0,0,0,0,0,0,0,0,0,0
1,308,4,Get Shorty (1995),5,209,1995,1,1998,2,6,...,0,0,0,0,0,0,0,0,0,0
2,308,7,Twelve Monkeys (1995),4,392,1995,1,1998,2,6,...,0,0,0,0,0,0,1,0,0,0
3,308,8,Babe (1995),5,219,1995,1,1998,2,6,...,0,0,0,0,0,0,0,0,0,0
4,308,9,Dead Man Walking (1995),4,299,1995,1,1998,2,6,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65003,729,346,Jackie Brown (1997),1,126,1997,1,1998,4,1,...,0,0,0,0,0,0,0,0,0,0
65004,729,690,Seven Years in Tibet (1997),2,155,1997,1,1998,4,1,...,0,0,0,0,0,0,0,0,1,0
65005,729,748,"Saint, The (1997)",4,316,1997,3,1998,4,1,...,0,0,0,0,0,1,0,1,0,0
65006,729,751,Tomorrow Never Dies (1997),3,180,1997,1,1998,4,1,...,0,0,0,0,0,1,0,1,0,0


### 발생한 문제

In [ ]:
df.movie_id.nunique(), df.movie_title.nunique()

(342, 338)

In [ ]:
df_m = df[['movie_id', 'movie_title']]
df_m.drop_duplicates(subset=['movie_id']).sort_values(['movie_title'], ascending=True)
#중복값 소환 시 실제 중복되는 것이 보이지 않음에도, movie id의 nunique는 342, title의 nunique는 338로 나온다. title에 null값은 없다.

,movie_id,movie_title
407,225,101 Dalmatians (1996)
80,178,12 Angry Men (1957)
59,135,2001: A Space Odyssey (1968)
139,291,Absolute Power (1997)
71,164,"Abyss, The (1989)"
...,...,...
64,151,Willy Wonka and the Chocolate Factory (1971)
56,132,"Wizard of Oz, The (1939)"
73,169,"Wrong Trousers, The (1993)"
104,208,Young Frankenstein (1974)


In [ ]:
df_m.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65008 entries, 0 to 65007
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movie_id     65008 non-null  int64 
 1   movie_title  65008 non-null  object
dtypes: int64(1), object(1)
memory usage: 1015.9+ KB


In [ ]:
# 최초 데이터 재확인
column_names = ['movie id', 'movie title', 'release date', 'video release date',
              'IMDb URL', 'unknown', 'Action', 'Adventure', 'Animation', 
              "Children's", "Comedy", 'Crime', 'Documentary', 'Drama', 'Fantasy', 
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
              'Thriller', 'War', 'Western']
movies = pd.read_csv('ml-100k/u.item', encoding='latin-1', sep='\|', header=None, index_col=0, names=column_names )

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1682 entries, 1 to 1682
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie title         1682 non-null   object 
 1   release date        1681 non-null   object 
 2   video release date  0 non-null      float64
 3   IMDb URL            1679 non-null   object 
 4   unknown             1682 non-null   int64  
 5   Action              1682 non-null   int64  
 6   Adventure           1682 non-null   int64  
 7   Animation           1682 non-null   int64  
 8   Children's          1682 non-null   int64  
 9   Comedy              1682 non-null   int64  
 10  Crime               1682 non-null   int64  
 11  Documentary         1682 non-null   int64  
 12  Drama               1682 non-null   int64  
 13  Fantasy             1682 non-null   int64  
 14  Film-Noir           1682 non-null   int64  
 15  Horror              1682 non-null   int64  
 16  Musica

In [ ]:
movies['movie title'].nunique()

1664

In [ ]:
movies[movies['movie title'].duplicated()].sort_values(['movie title'],ascending=True) 
#중복값 소환 시 실제 중복되는 것이 보이지 않음에도, movie id는 1682개, title의 nunique는 1664로 나온다. null값은 없다.

### 필요데이터 선별

In [ ]:
# 영화 id와 장르19개
df_movies = df[['movie_id', 'movie_title', 'unknown', 'Action',
       'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]

df_movie = df_movies.sort_values(['movie_id'], ascending=True).drop_duplicates(subset=['movie_id'])
df_movie = df_movie.reset_index(drop=True)
df_movie

,movie_id,movie_title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,7,Twelve Monkeys (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,8,Babe (1995),0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,928,"Craft, The (1996)",0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
338,1012,Private Parts (1997),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
339,1016,Con Air (1997),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
340,1028,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
# 인덱스 - 영화 id - 영화제목
index_movie = df_movie[['movie_id', 'movie_title']]
index_movie

,movie_id,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,4,Get Shorty (1995)
3,7,Twelve Monkeys (1995)
4,8,Babe (1995)
...,...,...
337,928,"Craft, The (1996)"
338,1012,Private Parts (1997)
339,1016,Con Air (1997)
340,1028,Grumpier Old Men (1995)


#### 현재 rating 갯수가 100개 이상인 데이터만 사용 중이나, 만약 전체 데이터 사용 시 참고할 부분

In [ ]:
# movie_id별 장르 정보 (데이터 거르지 않고 전체 쓸 때 사용 가능)
# g_vectors = movies[['unknown', 'Action', 'Adventure', 'Animation', 
#               "Children's", "Comedy", 'Crime', 'Documentary', 'Drama', 'Fantasy', 
#               'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
#               'Thriller', 'War', 'Western']]
# g_vectors

In [ ]:
# # 전체 데이터 사용 시
# # 인덱스는 0부터 시작, movie_id는 1부터 시작
# # idx및 column 재설정
# print(sim_g_vec.index)
# print('='*8)
# print(sim_g_vec.columns)
# print('='*8)
# sim_g_vec.index = sim_g_vec.index+1
# sim_g_vec.columns = sim_g_vec.columns+1
# sim_g_vec

### genre 기반

In [ ]:
genre_vec = df_movie[['unknown', 'Action',
       'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']]
genre_vec

,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0
338,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
339,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
340,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0


In [ ]:
# genre_vec 사이 유사도
from sklearn.metrics.pairwise import cosine_similarity
sim_g_vec = pd.DataFrame(cosine_similarity(genre_vec, genre_vec))
sim_g_vec

,0,1,2,3,4,5,6,7,8,9,...,332,333,334,335,336,337,338,339,340,341
0,1.000000,0.000000,0.333333,0.000000,0.666667,0.000000,0.000000,0.000000,0.577350,0.000000,...,0.000000,0.408248,0.000000,0.000000,0.577350,0.000000,0.408248,0.000000,0.408248,0.577350
1,0.000000,1.000000,0.333333,0.000000,0.000000,0.000000,0.408248,0.408248,0.000000,0.000000,...,0.000000,0.000000,0.666667,0.408248,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
2,0.333333,0.333333,1.000000,0.408248,0.666667,0.577350,0.000000,0.000000,0.577350,0.408248,...,0.577350,0.408248,0.333333,0.000000,0.577350,0.408248,0.816497,0.333333,0.408248,0.577350
3,0.000000,0.000000,0.408248,1.000000,0.408248,0.707107,0.000000,0.000000,0.000000,0.500000,...,0.707107,0.000000,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.000000
4,0.666667,0.000000,0.666667,0.408248,1.000000,0.577350,0.000000,0.000000,0.577350,0.408248,...,0.577350,0.408248,0.000000,0.000000,0.577350,0.408248,0.816497,0.000000,0.408248,0.577350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337,0.000000,0.000000,0.408248,0.500000,0.408248,0.707107,0.000000,0.000000,0.000000,0.500000,...,0.707107,0.000000,0.000000,0.500000,0.000000,1.000000,0.500000,0.000000,0.000000,0.000000
338,0.408248,0.000000,0.816497,0.500000,0.816497,0.707107,0.000000,0.000000,0.707107,0.500000,...,0.707107,0.500000,0.000000,0.000000,0.707107,0.500000,1.000000,0.000000,0.500000,0.707107
339,0.000000,1.000000,0.333333,0.000000,0.000000,0.000000,0.408248,0.408248,0.000000,0.000000,...,0.000000,0.000000,0.666667,0.408248,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
340,0.408248,0.000000,0.408248,0.000000,0.408248,0.000000,0.000000,0.000000,0.707107,0.500000,...,0.000000,1.000000,0.000000,0.000000,0.707107,0.000000,0.500000,0.000000,1.000000,0.707107


In [ ]:
sim_g_vec[0]

0      1.000000
1      0.000000
2      0.333333
3      0.000000
4      0.666667
         ...   
337    0.000000
338    0.408248
339    0.000000
340    0.408248
341    0.577350
Name: 0, Length: 342, dtype: float64

In [ ]:
# CB_genre_based

idx = np.argsort(sim_g_vec[0])[::-1][1:11] 
# index==0번 영화와 유사한 장르의 영화 10개 뽑아 idx에 넣기
# np.argsort() 정렬한 인덱스 반환, 오름차순
# sim_g_vec[0]이 오름차순으로 정렬되었기에 [::-1]역순출력 [시작:종료:규칙(방향)]
# [1:11]은 [0] 본인 제외한 10개

sim_g_vec[0].iloc[idx]
# 해당 lst 영화들과 0번 영화의 장르 유사도 확인

52     0.866025
51     0.816497
232    0.816497
165    0.816497
155    0.816497
138    0.816497
91     0.816497
152    0.816497
48     0.666667
294    0.666667
Name: 0, dtype: float64

In [ ]:
np.argsort(sim_g_vec[0]).max()

341

### recommender

In [ ]:
# recommender(movie_id, num of recommendation)
def recommender_genre(movie_id, num):

  # movie_id -> index
  indx = pd.Index(index_movie['movie_id']).get_loc(movie_id)
  
  # indx로 sim_g_vec 접근
  idx = np.argsort(sim_g_vec[indx])[::-1][1:num+1] 
      # index==0번 영화와 유사한 장르의 영화 10개 뽑아 idx에 넣기
      # np.argsort() 정렬한 인덱스 반환, 오름차순
      # sim_g_vec[0]이 오름차순으로 정렬되었기에 [::-1]역순출력 [시작:종료:규칙(방향)]
      # [1:11]은 [0] 본인 제외한 10개

  # index -> movie_id, movie_title 접근
  print('Movie Recommendation: ')
  print(index_movie.iloc[idx])


In [ ]:
recommender_genre(1047, 5)

Movie Recommendation: 
     movie_id              movie_title
154       242             Kolya (1996)
245       433          Heathers (1989)
242       428  Harold and Maude (1971)
235       410           Kingpin (1996)
229       393    Mrs. Doubtfire (1993)
